# Container

Create a file `app.py` with the Python command line application:

In [4]:
cat << EOF > app.py
import os
import click
import pystac
import rasterio
from rasterio.mask import mask
from pyproj import Transformer
from shapely import box
from loguru import logger


def aoi2box(aoi):
    """Converts an area of interest expressed as a bounding box to a list of floats"""
    return [float(c) for c in aoi.split(",")]


def get_asset(item, common_name):
    """Returns the asset of a STAC Item defined with its common band name"""
    for _, asset in item.get_assets().items():
        if not "data" in asset.to_dict()["roles"]:
            continue

        eo_asset = pystac.extensions.eo.AssetEOExtension(asset)
        if not eo_asset.bands:
            continue
        for b in eo_asset.bands:
            if (
                "common_name" in b.properties.keys()
                and b.properties["common_name"] == common_name
            ):
                return asset


@click.command(
    short_help="Crop",
    help="Crops a STAC Item asset defined with its common band name",
)
@click.option(
    "--input-item",
    "item_url",
    help="STAC Item URL or staged STAC catalog",
    required=True,
)
@click.option(
    "--aoi",
    "aoi",
    help="Area of interest expressed as a bounding box",
    required=True,
)
@click.option(
    "--epsg",
    "epsg",
    help="EPSG code",
    required=True,
)
@click.option(
    "--band",
    "band",
    help="Common band name",
    required=True,
)
def crop(item_url, aoi, band, epsg):

    if os.path.isdir(item_url):
        catalog = pystac.read_file(os.path.join(item_url, "catalog.json"))
        item = next(catalog.get_items())
    else:
        item = pystac.read_file(item_url)

    logger.info(f"Read {item.id} from {item.get_self_href()}")

    asset = get_asset(item, band)
    logger.info(f"Read asset {band} from {asset.get_absolute_href()}")

    if not asset:
        msg = f"Common band name {band} not found in the assets"
        logger.error(msg)
        raise ValueError(msg)

    bbox = aoi2box(aoi)

    with rasterio.open(asset.get_absolute_href()) as src:

        transformer = Transformer.from_crs(epsg, src.crs, always_xy=True)

        minx, miny = transformer.transform(bbox[0], bbox[1])
        maxx, maxy = transformer.transform(bbox[2], bbox[3])

        transformed_bbox = box(minx, miny, maxx, maxy)

        logger.info(f"Crop {asset.get_absolute_href()}")

        out_image, out_transform = rasterio.mask.mask(
            src, [transformed_bbox], crop=True
        )
        out_meta = src.meta.copy()

        out_meta.update(
            {
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "dtype": "uint16",
                "driver": "COG",
                "tiled": True,
                "compress": "lzw",
                "blockxsize": 256,
                "blockysize": 256,
            }
        )

        with rasterio.open(f"crop_{band}.tif", "w", **out_meta) as dst_dataset:
            logger.info(f"Write crop_{band}.tif")
            dst_dataset.write(out_image)

    logger.info("Done!")


if __name__ == "__main__":
    crop()
EOF

Create a file called `Dockerfile` with all the necessary commands to assemble an image

In [5]:
cat << EOF  > Dockerfile
FROM docker.io/python:3.10-slim

RUN pip install --no-cache-dir rasterio click pystac loguru pyproj shapely && \
    python -c "import rasterio"

ADD app.py /app/app.py

ENTRYPOINT []
EOF

In [6]:
cat Dockerfile

FROM docker.io/python:3.10-slim

RUN pip install --no-cache-dir rasterio click pystac loguru pyproj shapely &&     python -c "import rasterio"

ADD app.py /app/app.py

ENTRYPOINT []


Use `podman` to build the container:

In [7]:
podman build -t crop .

WARN[0000] "/" is not a shared mount, this could cause issues or missing mounts with rootless containers 
STEP 1/4: FROM docker.io/python:3.10-slim
Trying to pull docker.io/library/python:3.10-slim...
Getting image source signatures
Copying blob 4a654a9b63a6 [==============>---------------------] 1.3MiB / 3.2MiB
Copying blob 9e59167aa400 [====================================] 244.0b / 244.0b
Copying blob 1103112ebfc4 [=================>------------------] 1.7MiB / 3.3MiB
Copying blob d155c15b7553 [===>-------------------------------] 1.3MiB / 11.8MiB
Copying blob 8a1e25ce7c4f [>----------------------------------] 1.0MiB / 27.8MiB
Copying blob 4a654a9b63a6 [=======================>------------] 2.2MiB / 3.2MiB
Copying blob 9e59167aa400 done  
Copying blob 1103112ebfc4 [============================>-------] 2.7MiB / 3.3MiB
Copying blob d155c15b7553 [=======>---------------------------] 2.5MiB / 11.8MiB
Copying blob 8a1e25ce7c4f [==>--------------------------------] 2.5MiB / 27.8MiB
Copyi

List the container images:

In [8]:
podman images

REPOSITORY                TAG         IMAGE ID      CREATED         SIZE
localhost/crop            latest      31937611bd5c  36 seconds ago  325 MB
docker.io/library/python  3.10-slim   af6a90a1d65e  13 days ago     133 MB


Run the container to show the `crop` command line tool help:

In [11]:
podman run --rm -it -e PYTHONPATH=/app localhost/crop:latest python -m app --help

Usage: app.py [OPTIONS]

  Crops a STAC Item asset defined with its common band name

Options:
  --input-item TEXT  STAC Item URL or staged STAC catalog  [required]
  --aoi TEXT         Area of interest expressed as a bounding box  [required]
  --epsg TEXT        EPSG code  [required]
  --band TEXT        Common band name  [required]
  --help             Show this message and exit.


Run the application with arguments:

In [12]:
podman run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=.,target=/runs \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/crop:latest \
    python \
    -m \
    app \
    --aoi \
    "-121.399,39.834,-120.74,40.472" \
    --band \
    green \
    --epsg \
    "EPSG:4326" \
    --input-item \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A

2024-04-04 09:49:31.337 | INFO     | __main__:crop:69 - Read S2B_10TFK_20210713_0_L2A from https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
2024-04-04 09:49:31.959 | INFO     | __main__:crop:72 - Read asset green from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-04-04 09:49:33.988 | INFO     | __main__:crop:90 - Crop https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-04-04 09:49:59.952 | INFO     | __main__:crop:112 - Write crop_green.tif
2024-04-04 09:50:04.139 | INFO     | __main__:crop:115 - Done!
